<div >
<img src = "../banner.jpg" />
</div>

# Classification (Cont.)

To work through the steps of probability-based classification, we’ll use a real dataset on loans and credit from a set of local lenders in Germany (taken from the UC Irvine Machine Learning Repository and cleaned for our purposes). 

Credit scoring is a classic problem of classification, and it remains one of the big application domains for ML: use previous loan results (default versus payment) to train a model that can predict the performance of potential new loans.

\begin{align}
Default=f(x) + u
\end{align}

where $Default=I(Default=1)$

Taken from the UC Irvine Machine Learning Repository.


In [1]:
#Cargar librerías 
require("pacman")
p_load(tidyverse)
set.seed(1011)

Loading required package: pacman



In [2]:
#Leer los datos 
credit <- readRDS(url("https://github.com/ignaciomsarmiento/datasets/blob/main/credit_class.rds?raw=true"))
head(credit)   


,Default,duration,amount,installment,age,history,purpose,foreign,rent
,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>
1,0,6,1169,4,67,terrible,goods/repair,foreign,FALSE
2,1,48,5951,2,22,poor,goods/repair,foreign,FALSE
3,0,12,2096,2,49,terrible,edu,foreign,FALSE
4,0,42,7882,2,45,poor,goods/repair,foreign,FALSE
5,1,24,4870,3,53,poor,newcar,foreign,FALSE
6,0,36,9055,2,35,poor,edu,foreign,FALSE


In [3]:
default<-credit$Default  #defino ahora va a servir después

#mutación de factores
credit<-credit %>% mutate(Default=factor(Default,levels=c(0,1),labels=c("No","Si")),
                          history=factor(history,levels=c("good","poor","terrible"),labels=c("buena","mala","terrible")),
                          foreign=factor(foreign,levels=c("foreign","german"),labels=c("extranjero","aleman")),
                          purpose=factor(purpose,levels=c("newcar","usedcar","goods/repair","edu", "biz" ),labels=c("auto_nuevo","auto_usado","bienes","educacion","negocios")))         

In [4]:
mylogit <- glm(Default~., data = credit, family = "binomial")

In [5]:
pred<-predict(mylogit,newdata = credit, type = "response")


In [6]:
## what are our misclassification rates?
rule <- 1/2 # Bayes Rule

sum( (pred>rule)[default==0] )/sum(default==0) #False positive rate

[1] 0.08714286

In [7]:
sum( (pred>rule)[default==1] )/sum(default==1) #True positive rate

[1] 0.3

## Aside: Dummy Vars

In [8]:
p_load("caret")
dmy <- dummyVars(" ~ .", data = credit) # One-hot-encoding

credit <- data.frame(predict(dmy, newdata = credit))

## Out of sample prediction

In [9]:
credit<- credit  %>% mutate(Default=factor(Default.Si,levels=c(0,1),labels=c("No","Si")))

In [10]:
inTrain <- createDataPartition(
  y = credit$Default.Si,## La variable dependiente u objetivo 
  p = .7, ## Usamos 70%  de los datos en el conjunto de entrenamiento 
  list = FALSE)


train <- credit[ inTrain,]
test  <- credit[-inTrain,]

In [11]:
head(train)

,Default.No,Default.Si,duration,amount,installment,age,history.buena,history.mala,history.terrible,purpose.auto_nuevo,purpose.auto_usado,purpose.bienes,purpose.educacion,purpose.negocios,foreign.extranjero,foreign.aleman,rent.NA,rent.FALSE,rent.TRUE,Default
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,0,1,48,5951,2,22,0,1,0,0,0,1,0,0,1,0,0,1,0,Si
3,1,0,12,2096,2,49,0,0,1,0,0,0,1,0,1,0,0,1,0,No
5,0,1,24,4870,3,53,0,1,0,1,0,0,0,0,1,0,0,1,0,Si
6,1,0,36,9055,2,35,0,1,0,0,0,0,1,0,1,0,0,1,0,No
9,1,0,12,3059,2,61,0,1,0,0,0,1,0,0,1,0,0,1,0,No
11,0,1,12,1295,3,25,0,1,0,1,0,0,0,0,1,0,0,0,1,Si


### Logit

In [12]:
ctrl<- trainControl(method = "cv",
                    number = 5,
                    classProbs = TRUE,
                    verbose=FALSE,
                    savePredictions = T)


In [13]:
set.seed(1410)
mylogit_caret <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = train, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial")


mylogit_caret

Generalized Linear Model 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 561, 559, 560, 560 
Resampling results:

  Accuracy   Kappa  
  0.7185148  0.14365


In [14]:
predictTest_logit <- data.frame(
  obs = test$Default,                                    ## observed class labels
  predict(mylogit_caret, newdata = test, type = "prob"),         ## predicted class probabilities
  pred = predict(mylogit_caret, newdata = test, type = "raw")    ## predicted class labels
)


In [15]:
head(predictTest_logit)

,obs,No,Si,pred
,<fct>,<dbl>,<dbl>,<fct>
1,No,0.9495503,0.05044975,No
4,No,0.6613619,0.33863813,No
7,No,0.8174441,0.18255586,No
8,No,0.7748455,0.22515449,No
10,Si,0.6093878,0.39061224,No
14,Si,0.8519662,0.14803376,No


In [16]:
twoClassSummary(data = predictTest_logit, lev = levels(predictTest_logit$obs))

ROC      Sens      Spec 
0.7340984 0.9278351 0.2924528

In [17]:
set.seed(1410)
mylogit_en <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = train, 
                       method = "glmnet",
                       trControl = ctrl,
                       family = "binomial",
                        tuneGrid =expand.grid(alpha =seq(0,1,0.1),lambda = seq(0.1,.2,length.out =20)))


mylogit_en

glmnet 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 561, 559, 560, 560 
Resampling results across tuning parameters:

  alpha  lambda     Accuracy   Kappa       
  0.0    0.1000000  0.7156985   0.050330688
  0.0    0.1052632  0.7200047   0.054686564
  0.0    0.1105263  0.7200047   0.050659139
  0.0    0.1157895  0.7228824   0.056546919
  0.0    0.1210526  0.7271681   0.060851179
  0.0    0.1263158  0.7300050   0.066642216
  0.0    0.1315789  0.7300050   0.066642216
  0.0    0.1368421  0.7300050   0.066642216
  0.0    0.1421053  0.7285866   0.059825903
  0.0    0.1473684  0.7300050   0.058312074
  0.0    0.1526316  0.7300050   0.058312074
  0.0    0.1578947  0.7285765   0.051045750
  0.0    0.1631579  0.7285765   0.051045750
  0.0    0.1684211  0.7300153   0.054090181
  0.0    0.1736842  0.7300153   0.054090181
  0.0    0.1789474  0.7300153   0.054090181
  0.0    0.1842105  0.7314439   0.05684

## KNN

In [18]:
set.seed(1410)
mylogit_knn <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = train, 
                       method = "knn",
                       trControl = ctrl,
                     tuneGrid = expand.grid(k=c(3,5,7,9,11)))


mylogit_knn

k-Nearest Neighbors 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 561, 559, 560, 560 
Resampling results across tuning parameters:

  k   Accuracy   Kappa     
   3  0.6628908  0.07513750
   5  0.6729012  0.06729718
   7  0.6786157  0.02996589
   9  0.7042288  0.08218383
  11  0.7013816  0.05927847

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 9.

## LDA


\begin{align}
p (Y=1|X)=\frac{f(X|Y=1)p(Y=1)}{m(X)}
\end{align}


with $m(X)$ is the marginal distribution of $X$, i.e.

\begin{align}
m(X)=\int f(X|Y=y)p(Y=y)dy
\end{align}

Recall that there are two states of nature $y \rightarrow i\in\{0,1\}$


\begin{align}
m(X) &= f(X|Y=1)p(Y=1) + f(X|Y=0)p(Y=0) 
\end{align}


\begin{align}
m(X)     &= f(X|Y=1)p(Y=1) + f(X|Y=0)(1-p(Y=1))
\end{align}

We need to estimate $f(X|Y=1)$,  $f(X|Y=0)$ and $p(Y=1)$ 


#### By Hand


- Let's start by estimating $p(Y=1)$. We've done this before

    \begin{align}
    p(Y=1) = \frac{\sum_{i=1}^n 1[Y_i=1]}{N}
    \end{align}


In [19]:
p1<-sum(train$Default.Si)/dim(train)[1]
p1

[1] 0.2771429


- Next $f(X|Y=j)$ with $j=0,1$. 

    - If we assume one predictor and $X|Y\sim N(\mu_j,\sigma_j)$, the problem boils down to estimating $\mu_j,\sigma_j$

    - LDA makes it simpler, assumes $\sigma_j=\sigma$ $\forall j$

To do this partition the sample in two $Y=0$ and $Y=1$, estimate the moments and get $\hat{f}(X|Y=j)$

**Means**

\begin{align}
\hat{\mu}_k=\frac{1}{n_k}\sum_{i:y_i=k}x_i
\end{align}

In [20]:
#Means
mu1<-mean(train$duration[train$Default.Si==1])
mu1

[1] 23.92784

In [21]:
mu0<-mean(train$duration[train$Default.Si==0])
mu0

[1] 19.63834

**Variance**

\begin{align}
\hat{\sigma}^2 = \frac{1}{N-K} \sum_{k=1}^K \sum_{i:y_i=k} (x_i -\hat{\mu}_k)^2
\end{align}

In [22]:
#Variance
g1<-sum((train$duration[train$Default.Si==1]-mu1)^2)
g0<-sum((train$duration[train$Default.Si==0]-mu0)^2)


sigma<-sqrt((g1+g0)/(dim(train)[1]-2))
sigma

[1] 11.72902

With the moments, now we can obtain $f(X|Y=j)$ with $j=0,1$. 

In [23]:
f1<-dnorm(test$duration,mean=mu1,sd=sigma)
f0<-dnorm(test$duration,mean=mu0,sd=sigma)

- Finally plug everything into the Bayes Rule and we are done:
\begin{align}
p (Y=1|X)=\frac{f(X|Y=1)p(Y=1)}{f(X|Y=1)p(Y=1) + f(X|Y=0)(1-p(Y=1))}
\end{align}


In [24]:
post_hand<-f1*p1/(f1*p1+f0*(1-p1))
head(post_hand)

[1] 0.1898781 0.4186529 0.2912018 0.3739312 0.3312645 0.2912018

In [25]:
p_load("MASS")     # LDA
lda_simple <- lda(Default.Si~duration, data = train)
lda_simple_pred<-predict(lda_simple,test)
names(lda_simple_pred)


[1] "class"     "posterior" "x"

In [26]:
posteriors<-data.frame(lda_simple_pred$posterior)
posteriors$hand<-post_hand

head(posteriors)

,X0,X1,hand
,<dbl>,<dbl>,<dbl>
1,0.8101219,0.1898781,0.1898781
4,0.5813471,0.4186529,0.4186529
7,0.7087982,0.2912018,0.2912018
8,0.6260688,0.3739312,0.3739312
10,0.6687355,0.3312645,0.3312645
14,0.7087982,0.2912018,0.2912018


### Caret

In [27]:
lda_fit = train(Default~duration+amount+installment+age, 
                data=train, 
                method="lda",
                trControl = ctrl)

lda_fit

Linear Discriminant Analysis 

700 samples
  4 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 560, 560, 561, 559 
Resampling results:

  Accuracy   Kappa     
  0.7200351  0.02909477


In [28]:
head(credit)

,Default.No,Default.Si,duration,amount,installment,age,history.buena,history.mala,history.terrible,purpose.auto_nuevo,purpose.auto_usado,purpose.bienes,purpose.educacion,purpose.negocios,foreign.extranjero,foreign.aleman,rent.NA,rent.FALSE,rent.TRUE,Default
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,1,0,6,1169,4,67,0,0,1,0,0,1,0,0,1,0,0,1,0,No
2,0,1,48,5951,2,22,0,1,0,0,0,1,0,0,1,0,0,1,0,Si
3,1,0,12,2096,2,49,0,0,1,0,0,0,1,0,1,0,0,1,0,No
4,1,0,42,7882,2,45,0,1,0,0,0,1,0,0,1,0,0,1,0,No
5,0,1,24,4870,3,53,0,1,0,1,0,0,0,0,1,0,0,1,0,Si
6,1,0,36,9055,2,35,0,1,0,0,0,0,1,0,1,0,0,1,0,No


## Naive Bayes

In [43]:
p_load("klaR")

set.seed(1410)
mylogit_nb <- train(Default~duration+amount+installment+age+
                       history.buena+history.mala+
                       purpose.auto_nuevo+purpose.auto_usado+purpose.bienes+purpose.educacion+
                       foreign.extranjero+
                       +rent.TRUE, 
                       data = train, 
                       method = "nb",
                       trControl = ctrl,
                        tuneGrid=expand.grid(fL=seq(0,10,length.out = 3),
                                            usekernel=TRUE,
                                            adjust=seq(1,10,length.out = 3)))



mylogit_nb

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 1”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 2”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 3”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 4”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 5”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 6”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 7”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 8”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 9”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 10”
Warning message in 

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 84”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 85”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 86”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 87”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 88”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 89”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 90”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 91”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 92”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 93”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 27”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 28”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 29”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 30”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 31”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 32”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 33”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 34”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 35”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 36”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 110”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 111”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 112”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 113”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 114”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 115”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 116”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 117”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 118”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 119”


Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 53”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 54”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 55”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 56”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 57”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 58”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 59”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 60”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 61”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 62”
Warning me

...

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 137”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 138”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 139”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 140”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 1”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 2”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 3”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 4”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 5”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 6”
Warning mess

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 80”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 81”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 82”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 83”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 84”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 85”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 86”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 87”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 88”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 89”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 23”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 24”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 25”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 26”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 27”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 28”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 29”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 30”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 31”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 32”
Warning me

Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 106”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 107”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 108”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 109”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 110”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 111”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 112”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 113”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 114”
Warning message in FUN(X[[i]], ...):
“Numerical 0 probability for all classes with observation 115”


Naive Bayes 

700 samples
 12 predictor
  2 classes: 'No', 'Si' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 560, 561, 559, 560, 560 
Resampling results across tuning parameters:

  fL  adjust  Accuracy   Kappa      
   0   1.0    0.7214128  0.001666648
   0   5.5    0.7271781  0.043561357
   0  10.0    0.7228617  0.000000000
   5   1.0    0.7214128  0.001666648
   5   5.5    0.7271781  0.043561357
   5  10.0    0.7228617  0.000000000
  10   1.0    0.7214128  0.001666648
  10   5.5    0.7271781  0.043561357
  10  10.0    0.7228617  0.000000000

Tuning parameter 'usekernel' was held constant at a value of TRUE
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were fL = 0, usekernel = TRUE and adjust
 = 5.5.